In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
import re
import pandas as pd
from textblob import TextBlob

def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = '8XO4C13ctzGmaXleZT2fczRbv'
    consumer_secret = 'LiaXSL9OnO1gkEppEYWVWCUQTqwJ9ba8sW64Pc52rixpfWhYO9'
    access_token = '915301601987919872-ihW5x7SV8dMW5lAG0nbyi69YfnmqQQl'
    access_secret = 'xfz573YVAzIdSbMSlF5ffxLhzp6laG11vCfgCUoyWiRLd'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

In [2]:
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

In [3]:
def get_tweet_id(api, date='', days_ago=9, query='a'):
    ''' Function that gets the ID of a tweet. This ID can then be
        used as a 'starting point' from which to search. The query is
        required and has been set to a commonly used word by default.
        The variable 'days_ago' has been initialized to the maximum
        amount we are able to search back in time (9).'''
    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id

In [4]:
def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''

    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')

In [5]:
def main():
    ''' This is a script that continuously searches for tweets
        that were created over a given number of days. The search
        dates and search phrase can be changed below. '''


    ''' search variables: '''
    search_phrases = ['Verizon','verizon','#Verizon','#verizon']
    time_limit = 1.5                           # runtime limit in hours
    max_tweets = 100                           # number of tweets per search (will be
                                               # iterated over) - maximum is 100
    min_days_old, max_days_old = 0, 7          # search limits e.g., from 7 to 8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                               # states (and a large portion of Canada)
    

    # loop over search items,
    # creating a new file for each
    for search_phrase in search_phrases:

        print('Search phrase =', search_phrase)

        ''' other variables '''
        name = search_phrase.split()[0]
        json_file_root = name + '/'  + name
        os.makedirs(os.path.dirname(json_file_root), exist_ok=True)
        read_IDs = False
        
        # open a file in which to store the tweets
        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                  d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ',json_file)
            read_IDs = True
        
        # authorize and load the twitter API
        api = load_api()
        
        # set the 'starting point' ID for tweet collection
        if read_IDs:
            # open the json file and get the latest tweet ID
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        # set the smallest ID to search for
        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) =', max_id)
        print('since id (ending point) =', since_id)
        


        ''' tweet gathering loop  '''
        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                          max_id=max_id, since_id=since_id,
                                          geocode=USA)
            # write tweets to file in JSON format
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        sys.exit('Maximum number of empty tweet strings reached - exiting')
                    else:
                        print('Maximum number of empty tweet strings reached - breaking')
                        break


if __name__ == "__main__":
    main()

Search phrase = Verizon
search limit (start/stop): 2018-03-31 23:59:59
max id (starting point) = -1
since id (ending point) = 980233278052601856
count = 1
found 100 tweets
count = 2
found 93 tweets
found 7 tweets
count = 3
found 100 tweets
count = 4
found 100 tweets
count = 5
found 100 tweets
count = 6
found 100 tweets
count = 7
found 100 tweets
count = 8
found 95 tweets
found 5 tweets
count = 9
found 98 tweets
found 2 tweets
count = 10
found 100 tweets
count = 11
found 100 tweets
count = 12
found 100 tweets
count = 13
found 85 tweets
found 13 tweets
found 2 tweets
count = 14
found 100 tweets
count = 15
found 100 tweets
count = 16
found 100 tweets
count = 17
found 76 tweets
found 7 tweets
found 9 tweets
found 5 tweets
found 1 tweets
found 1 tweets
found 1 tweets
count = 18
found 43 tweets
found 37 tweets
found 19 tweets
found 1 tweets
count = 19
found 93 tweets
found 6 tweets
found 0 tweets
no tweets found
count = 20
found 99 tweets
found 0 tweets
no tweets found
count = 21
found 95 tw

found 100 tweets
count = 236
found 100 tweets
count = 237
found 100 tweets
count = 238
found 100 tweets
count = 239
found 100 tweets
count = 240
found 100 tweets
count = 241
found 100 tweets
count = 242
found 100 tweets
count = 243
found 100 tweets
count = 244
found 100 tweets
count = 245
found 100 tweets
count = 246
found 100 tweets
count = 247
found 100 tweets
count = 248
found 100 tweets
count = 249
found 100 tweets
count = 250
found 100 tweets
count = 251
found 100 tweets
count = 252
found 100 tweets
count = 253
found 100 tweets
count = 254
found 100 tweets
count = 255
found 100 tweets
count = 256
found 100 tweets
count = 257
found 100 tweets
count = 258
found 100 tweets
count = 259
found 100 tweets
count = 260
found 100 tweets
count = 261
found 100 tweets
count = 262
found 100 tweets
count = 263
found 100 tweets
count = 264
found 100 tweets
count = 265
found 100 tweets
count = 266
found 98 tweets
found 2 tweets
count = 267
found 100 tweets
count = 268
found 100 tweets
count = 269


SystemExit: Maximum number of empty tweet strings reached - exiting

C:\Users\sugan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
import json

In [28]:
tweetfiles = ['Verizon_2018-04-01_to_2018-04-07.json']
tweets=[]

In [29]:
for files in tweetfiles:
    with open(files,'r') as f:
        for line in f.readlines():
            tweets.append(json.loads(line))

In [42]:
tweets[1]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Apr 07 23:37:56 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 3087684921,
    'id_str': '3087684921',
    'indices': [0, 8],
    'name': 'wrists slit IE',
    'screen_name': 'jarkeem'},
   {'id': 971980688143532032,
    'id_str': '971980688143532032',
    'indices': [9, 19],
    'name': 'plant (black excellence v2)',
    'screen_name': 'plant0024'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 982764440276828161,
 'id_str': '982764440276828161',
 'in_reply_to_screen_name': 'jarkeem',
 'in_reply_to_status_id': 982760913068593153,
 'in_reply_to_status_id_str': '982760913068593153',
 'in_reply_to_user_id': 3087684921,
 'in_reply_to_user_id_str': '3087684921',
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http:/

In [30]:
def populate_tweet_df(tweets):
    df = pd.DataFrame()  
    df['tweet_id'] = list(map(lambda tweet: tweet['id'], tweets))
    df['username'] = list(map(lambda tweet:tweet['user']['name'],tweets))
    df['date_time'] = list(map(lambda tweet: tweet['created_at'], tweets))
    df['text'] = list(map(lambda tweet: tweet['text'], tweets))
    df['retweet'] = list(map(lambda tweet:True if 'retweeted_status' in tweet.keys() else False,tweets)) 
    df['retweet_count'] = list(map(lambda tweet:tweet['retweet_count'],tweets))
    df['user_location'] = list(map(lambda tweet: tweet['user']['location'], tweets)) 
    df['country_code'] = list(map(lambda tweet: tweet['place']['country_code']
                                  if tweet['place'] != None else '', tweets))
    df['long'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][0]
                        if tweet['coordinates'] != None else 'NaN', tweets))
    df['latt'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][1]
                        if tweet['coordinates'] != None else 'NaN', tweets)) 
    return df

In [31]:
df = populate_tweet_df(tweets)

In [32]:
# ## To format the text
# de = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", df['text'][1]).split())

In [33]:
def sentiment_analysis(text):
    temp = TextBlob(text)
    if(temp.sentiment.polarity > 0):
        return "positive"
    elif(temp.sentiment.polarity < 0):
        return "negative"
    else:
        return "neutral"

In [34]:
df['sentiment'] = df['text'].apply(sentiment_analysis)

In [41]:
df.text

0        RT @IvankaTrump: Verizon will donate $400M tha...
1            @jarkeem @plant0024 I got Verizon too what Up
2        One of my students father passed away while bu...
3        RT @IvankaTrump: Verizon will donate $400M tha...
4        RT @gavincannonn: @Verizon your service fuckin...
5        Item specifics Condition: Used : An item that ...
6        Item specifics Condition: Used : An item that ...
7        iPhone X 256GB – Space Gray / Silver (Verizon)...
8        @ChetPowell Or kids,  just so dern sexy, poor ...
9          @TakumaSatoRacer @ISMRaceway @verizon がんばって下さい！
10       ROLL 2 - Atmosphere - Ultimate Ears https://t....
11       @iSmashFizzle This bothers me A LOT!!! https:/...
12       iPhone X 256GB - Space Gray / Silver (Verizon)...
13       Apple iPhone 6s Plus 32GB Smartphone (AT&amp;T...
14       RT @TakumaSatoRacer: Ready to race under the l...
15       Apple iPhone 6s Plus 32GB Smartphone (AT&amp;T...
16       @HuffPost Some kids get trafficked  https://t..

In [43]:
df.to_csv('test.csv')